### Data Merge - Online / Batch -
* Online Sample Data: 477420204194, 474620226651
<br/>
#### 할일
* STEP-01. 각 데이터 타입별 대상 레코드 및 컬럼 추출
* STEP-02. 공사비별 전주/전선/인입선 갯 수 계산

In [10]:
import os
import sys

import aidd.sys.config as cfg
from aidd.batch.data_manager import get_provide_data, read_data

In [2]:
pdata = get_provide_data()

[14c11ed044f1][2024-03-12 15:57:44.677404] 제공받은 데이터 불러오기 시작
[14c11ed044f1][2024-03-12 15:58:12.662500]   읽은 데이터: CONS, 데이터 크기: (19052, 143), 처리시간: 0:00:27.984072
[14c11ed044f1][2024-03-12 15:58:37.201544]   읽은 데이터: POLE, 데이터 크기: (38533, 63), 처리시간: 0:00:24.538868
[14c11ed044f1][2024-03-12 15:59:07.886384]   읽은 데이터: LINE, 데이터 크기: (40019, 77), 처리시간: 0:00:30.684695
[14c11ed044f1][2024-03-12 15:59:20.618733]   읽은 데이터: SL, 데이터 크기: (22632, 57), 처리시간: 0:00:12.732217
[14c11ed044f1][2024-03-12 15:59:20.618827] 제공받은 데이터 불러오기 종료, 최종 처리시간: 0:01:35.941438


In [16]:
# 공사비 학습대상 레코드
df = pdata['CONS']
## [전처리]: 조건 적용(이 조건을 만족하지 않으면 제외), 
##           단, 실시간 처리시 총 공사비는 제외
modeling_records = \
    (df.ACC_TYPE_NAME == cfg.COND_ACC_TYPE_NAME) & \
    (df.CONT_CAP < cfg.COND_MAX_CONT_CAP) & \
    (df.CONS_TYPE_CD == cfg.COND_CONS_TYPE_CD) & \
    (df.TOTAL_CONS_COST < cfg.COND_MAX_TOTAL_CONS_COST)
cons_df = df[modeling_records].reset_index(drop=True)
# cons_df = cons_df[cfg.CONS_COLS].copy()

AttributeError: module 'aidd.sys.config' has no attribute 'CONS_COLS'

In [9]:
## [전처리]: 별도 계산 필요
cons_df.fillna(0, inplace=True)

In [ ]:
## [전처리]: 별도 컬럼 추가
cons_df['YEAR'] = cons_df.LAST_MOD_DATE.dt.year
cons_df['MONTH'] = cons_df.LAST_MOD_DATE.dt.month
cons_df['DAY'] = cons_df.LAST_MOD_DATE.dt.day
cons_df['DAYOFWEEK'] = cons_df.LAST_MOD_DATE.dt.dayofweek
cons_df['DAYOFYEAR'] = cons_df.LAST_MOD_DATE.dt.dayofyear
cons_df['YEAR_MONTH'] = cons_df.LAST_MOD_DATE.dt.strftime('%Y%m').astype(int)


In [12]:
temp_df = read_data(pmode='PP', file_code='SL')

/home/freeman/projects/aidd/batch/data_manager.py:34: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, **kwargs)


In [13]:
temp_df['REAL_POLE_CNTS'].max(), temp_df['REAL_POLE_CNTS'].min()

(10, 1)

In [15]:
temp_df[temp_df['REAL_POLE_CNTS']==4]

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,EID_CODE,CONT_CAP,YEAR,MONTH,DAY,...,SL_SPEC_10.0,SL_SPEC_16.0,SL_SPEC_22.0,SL_SPEC_25.0,SL_SPEC_35.0,SL_SPEC_38.0,SL_SPEC_60.0,SL_SPEC_70.0,SL_SPEC_100.0,SL_SPEC_120.0
33,477420204092,7009709,2020-12-30 17:16:31,MDE1900011,단양지사,MDE,5,2020,12,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,477420204194,9076645,2020-12-23 11:50:39,MDE1900011,단양지사,MDE,3,2020,12,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,477420204310,7888323,2021-02-24 17:58:12,MDE1900011,단양지사,MDE,3,2021,2,24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,477420213019,8049521,2021-03-31 17:47:26,2018109107,단양지사,AAA,5,2021,3,31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131,477420213449,11653146,2021-07-02 17:07:51,MDP2100146,단양지사,MDP,3,2021,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11237,474620226651,9512744,2023-04-04 08:27:05,MDP2300252,충주지사,MDP,5,2023,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11299,474620233689,8746232,2023-04-26 12:59:54,MDP2300252,충주지사,MDP,3,2023,4,26,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11306,474620233752,7053178,2023-04-25 16:13:48,MDP2300252,충주지사,MDP,3,2023,4,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11329,474620233929,7110535,2023-05-19 09:52:27,MDP2300252,충주지사,MDP,5,2023,5,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
